In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers sentence-transformers googletrans==4.0.0-rc1
!pip install git+https://github.com/huggingface/transformers accelerate
!pip install qwen-vl-utils[decord]==0.0.8
!pip install --upgrade --no-cache-dir transformers
!pip install --upgrade timm
!pip install --upgrade accelerate
!pip install --upgrade git+https://github.com/huggingface/transformers.git


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=85788718e8a2b2d6e428459a328bdd093d7153ee1a0d44dddb7364605e9a0ab8
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
     

In [2]:
# Import required libraries
import time
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from googletrans import Translator
from huggingface_hub import login
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
# Authenticate with Hugging Face (needed for gated models)
login()

# Download NLTK tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Define Student Email (from Assignment Instructions)
email_text = """Subject: Concerns About Professor X’s Conduct

Dear Dr. Ustaad,
I hope this email finds you well. I am writing to express my concerns about Professor X’s conduct 
during the Introduction to Zoology class last semester. On multiple occasions, Professor X made 
dismissive remarks about students’ questions and failed to provide clear feedback on assignments.  

Additionally, the grading seemed inconsistent and unfair, with no opportunity for clarification or 
appeal. 

I found this experience deeply frustrating and demotivating, and I believe it affected my performance 
in the course. I would appreciate it if the department could look into this matter and ensure that 
future students have a more positive and supportive learning environment. 

Thank you for your attention to this matter. 
Sincerely, 
Shaagird"""

In [5]:
# Load the TowerInstruct-13B model
n3_model = "jpacifico/Chocolatine-2-14B-Instruct-v2.0b2"

In [ ]:
import torch

# Automatically determine the best device
device = "cuda" if torch.cuda.is_available() and torch.cuda.mem_get_info(0)[0] > 1e9 else "cpu"

# Load summarization model
summarizer = pipeline("summarization", model=n3_model,device=device)

# Load question answering model
qa_pipeline = pipeline("question-answering", model=n3_model,device_map="auto")

# For keyword extraction, use a separate model designed for sentence embeddings (e.g., 'all-MiniLM-L6-v2')
sbert_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)  # Consider using the best model for your purpose

# Initialize the Translator
translator = pipeline("translation_en_to_fr", model=n3_model,device_map="auto")  # English to French

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/47.5k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
translator_en_fr = pipeline("translation_en_to_fr", model=n3_model, device=0)
translator_fr_en = pipeline("translation_fr_to_en", model=n3_model, device=1)


In [ ]:
def measure_time(task_function, *args, model_name="Unknown", **kwargs):
    start = time.time()
    result = task_function(*args, **kwargs) 
    end = time.time()
    elapsed_time = end - start
    print(f"{model_name} Execution Time: {elapsed_time:.2f} seconds")
    return result, elapsed_time

execution_times = {}

In [ ]:
### **TASK 1: Summarization**
print("\nPerforming Summarization...")
summary_14b, execution_times["Summarization_14B"] = measure_time(
    summarizer, email_text, max_length=50, min_length=20, do_sample=False, model_name="Summarization (14B)"
)

# Extract the summarized text
summarized_text = summary_14b[0]['summary_text'] if isinstance(summary_14b, list) else summary_14b

In [ ]:
### **TASK 2: Question Answering**
print("\nPerforming Question Answering...")
question = "What was the reason for the student's disappointment with Professor X?"
qa_14b, execution_times["QA_14B"] = measure_time(qa_pipeline, question=question, context=email_text, model_name="Question Answering (14B)")

In [ ]:
### **TASK 3: Keyword Extraction (Based on Summarized Text)**
print("\nExtracting Keywords...")

from keybert import KeyBERT
import time

# Initialize KeyBERT model
kw_model = KeyBERT("all-MiniLM-L6-v2")  # Uses a small but powerful embedding model

# Start time tracking
start_time = time.time()

# Extract key phrases (n-grams) with KeyBERT
keywords = kw_model.extract_keywords(
    summarized_text, 
    keyphrase_ngram_range=(1, 3),  # Extract single words & short phrases
    stop_words="english",  # Remove common words
    top_n=10  # Extract top 10 keywords
)

# Extract just the words (not scores)
extracted_keywords = [kw[0] for kw in keywords]

# Record execution time
execution_times["Keyword_Extraction"] = time.time() - start_time

print(f"Keyword Extraction Execution Time: {execution_times['Keyword_Extraction']:.2f} seconds")
print(f"Extracted Keywords: {extracted_keywords}")

In [ ]:
### **TASK 4: Paragraph translation**
print("\nTranslating to french...")
french_translation, execution_times["Translation_FR"] = measure_time(
    translator_en_fr, email_text, max_length=500, model_name="Translation to French"
)
print("\nTranslating back to english...")
back_translation, execution_times["Translation_EN"] = measure_time(
    translator_fr_en, french_translation[0]['translation_text'], max_length=500, model_name="Translation back to English"
)


In [ ]:
# Print Results
print("\n--- RESULTS ---")
print("\nSummarization:")
print(f"14B Model: {summary_14b[0]['summary_text']}")

print("\nQuestion Answering:")
print(f"14B Model: {qa_14b['answer']}")

print(f"Keyword Extraction Execution Time: {execution_times['Keyword_Extraction']:.2f} seconds")
print(f"Extracted Keywords: {extracted_keywords}")

print("\nTranslation:")
print(f"French Translation: {french_translation[0]['translation_text']}")
print(f"Back to English Translation: {back_translation[0]['translation_text']}")

# Print Execution Times for Each Task
print("\n--- Execution Times (Seconds) ---")
for task, time_taken in execution_times.items():
    print(f"{task}: {time_taken:.2f} sec")